# Building a RAG system with OpenAI and ChromaDB

#### Introduction
Retrieval-Augmented Generation (RAG) is a powerful technique that combines the capabilities of large language models with external knowledge retrieval. This notebook will walk you through building a complete RAG system using:

- LangChain: A framework for developing applications powered by language models
- ChromaDB: An open-source vector database for storing and retrieving embeddings
- OpenAI: For embeddings and language model (you can substitute with other providers)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# langchain imports
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import (
    PyPDFLoader,
    PyMuPDFLoader,
    TextLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# vector stores
from langchain_community.vectorstores import Chroma

# Utilities
import numpy as np
from typing import List

## RAG Architecture Overview
### RAG (Retrieval-Augmented Generation) Architecture:

1. Document Loading: Load documents from various sources
2. Document Splitting: Break documents into smaller chunks
3. Embedding Generation: Convert chunks into vector representations
4. Vector Storage: Store embeddings in ChromaDB
5. Query Processing: Convert user query to embedding
6. Similarity Search: Find relevant chunks from vector store
7. Context Augmentation: Combine retrieved chunks with query
8. Response Generation: LLM generates answer using context

Benefits of RAG:
- Reduces hallucinations
- Provides up-to-date information
- Allows citing sources
- Works with domain-specific knowledge

### Create sample data

In [ ]:
sample_docs = [
    """
    Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    """,
    
    """
    Deep Learning and Neural Networks
    
    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning has revolutionized fields like computer vision, natural language 
    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly 
    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers 
    excel at sequential data processing.
    """,
    
    """
    Natural Language Processing (NLP)
    
    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context and relationships between words in text.
    """
]

In [ ]:
# save to file
import tempfile

temp_dir = tempfile.mkdtemp()

for i, doc in enumerate(sample_docs):
    with open(f"{temp_dir}/doc_{i}.txt", "w") as f:
        f.write(doc)

print(f"Sample document created: {temp_dir}")

### Document loading

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

In [ ]:
loader = DirectoryLoader(
    path=temp_dir,
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding':"utf-8"}
)

documents = loader.load()

print(f"Loaded {len(documents)} documents")
print(f"\nFirst document preview: ")
print(documents[0].page_content[:200] + "...")

### Document splitting

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len,
    separators=["\n\n", "\n", ". ", " ", ""] # heirarchy of separators
)

chunks = text_splitter.split_documents(documents=documents)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")
print(f"\nChunk example:")
print(f"Content: {chunks[0].page_content[:150]}...")
print(f"Metadata: {chunks[0].metadata}")

In [ ]:
chunks

### Embeddings models

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### Initialize chromadb vector store and store chunks in vector representation

In [ ]:
# create chromadb vector store
persist_dir = "./chroma_db"

# initialize chromadb with openai embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_dir,
    collection_name="rag_collection"
)

print(f"Vector store created with {vectorstore._collection.count()} vectors")
print(f"Persisted to: {persist_dir}")

### Test similarity search

In [ ]:
query = "What are the types of machine learning?"

similar_docs = vectorstore.similarity_search(query=query, k=3)
similar_docs

In [ ]:
print(f"Query: {query}")
print(f"\nTop {len(similar_docs)} similar chunks: ")
for i, doc in enumerate(similar_docs):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:200]+"...")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")

### Advanced similarity search with scores

In [ ]:
results = vectorstore.similarity_search_with_score(query, k=3)
results

#### Understanding Similarity Scores
The similarity score represents how closely related a document chunk is to your query. The scoring depends on the distance metric used:

ChromaDB default: Uses L2 distance (Euclidean distance)

- Lower scores = MORE similar (closer in vector space)
- Score of 0 = identical vectors
- Typical range: 0 to 2 (but can be higher)


Cosine similarity (if configured):

- Higher scores = MORE similar
- Range: -1 to 1 (1 being identical)

### Initialize the LLM (OpenAI), RAG chain, Prompt template, query the RAG system

In [ ]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.2,
    max_tokens=500
)

In [ ]:
test_response = llm.invoke("What is Large language models?")
test_response

In [ ]:
from langchain.chat_models.base import init_chat_model

llm = init_chat_model(
    model="openai:gpt-3.5-turbo"
)

# llm = init_chat_model(
#     model="groq:"
# )

test_response = llm.invoke("What is Large language models?")
test_response

### Modern RAG chain

In [ ]:
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [ ]:
## Convert vector store to retiever
retriever = vectorstore.as_retriever(
    search_kwarg = {'k':3} ## Retrieve top 3 relevant chunks
)
retriever

In [ ]:
## Create a prompt template
system_prompt = """
You are an assistant for question-answering tasks.
Use the following pieces of retrived context to answer the question.
If you dont know the answer, just say that you dont know.
Use three sentences maximum and keep the answer concise.

Context: {context}
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt),
     ("human", "{input}")]
)

In [ ]:
prompt.to_json()

### Create document chain

In [ ]:
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

This chain:

- Takes retrieved documents
- "Stuffs" them into the prompt's {context} placeholder
- Sends the complete prompt to the LLM
- Returns the LLM's response

#### What is create_retrieval_chain?
create_retrieval_chain is a function that combines a retriever (which fetches relevant documents) with a document chain (which processes those documents with an LLM) to create a complete RAG pipeline.

In [ ]:
rag_chain = create_retrieval_chain(retriever, document_chain)
rag_chain

In [ ]:
response = rag_chain.invoke({"input": "What is machine learning?"})
response

In [ ]:
response["answer"]

In [ ]:
# Function to query the modern RAG system
def query_rag_modern(question):
    print(f"Question: {question}")
    print("-" * 50)
    
    # Using create_retrieval_chain approach
    result = rag_chain.invoke({"input": question})
    
    print(f"Answer: {result['answer']}")
    print("\nRetrieved Context:")
    for i, doc in enumerate(result['context']):
        print(f"\n--- Source {i+1} ---")
        print(doc.page_content[:200] + "...")
    
    return result

# Test queries
test_questions = [
    "What are the three types of machine learning?",
    "What is deep learning and how does it relate to neural networks?",
    "What are CNNs best used for?"
]

for question in test_questions:
    result = query_rag_modern(question)
    print("\n" + "="*80 + "\n")

# Create RAG chain alternative: LCEL (LangChain Expression Language)

In [ ]:
# Even a more flexible approach using LECL
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# create a custom prompt
custom_prompt = ChatPromptTemplate.from_template("""Use the following context to answer the question.
                                                 If you dont know the answer based on the context, say you don't know.
                                                 Provide specific details from the context to support your answer.
                                                 
                                                 Context:
                                                 {context}

                                                 Question: {question}
                                                 
                                                 Answer:""")


In [ ]:
# Format the output documents for the prompt
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Build chain using LCEL
rag_chain_lcel = (
    {
        'context': retriever| format_docs,
        'question': RunnablePassthrough()
    }
     | custom_prompt
     | llm
     | StrOutputParser()
)
rag_chain_lcel

In [ ]:
rag_chain_lcel.invoke("What the hell is deep learning?")

In [ ]:
retriever.invoke("What is deep learning?")

In [ ]:
def query_rag_lcel(question):
    print(f"Question: {question}")
    print("-"*50)

    # method 1: pass string directly (when using RunnablePassThrough)
    answer = rag_chain_lcel.invoke(question)
    print(f"Answer: {answer}")

    # get source documents seperately if needed
    docs = retriever.invoke(question)
    print("\nSOurce Documents: ")
    for i, doc in enumerate(docs):
        print(f"\n--- Source {i+1} ---")
        print(doc.page_content[:200]+"...")

In [ ]:
# Test LCEL chain
query_rag_lcel("whats all this about?")

# Add New Documents to existing Vector Store

In [ ]:
new_document = """
Reinforcement Learning in Detail

Reinforcement learning (RL) is a type of machine learning where an agent learns to make
decisions by interacting with an environment. The agnet receives rewards or penalties
based on its actions and learns to maximize cumulative rewards over time. Key concepts
in RL include: states, actions, rewards, policies, and value functions. Popular RL 
algorithms include Q-learning, Deep Q-Networks (DQN), Policy gradient methods, and
actor-critic methods. RL has been sucessfully applied to game playing (like AlphaGo),
robotics, and autonomous systems.
"""

In [ ]:
new_doc = Document(
    page_content=new_document,
    metadata={
        "source":"manual_addition",
        "topic":"reinforcement learning"
    }
)
new_doc

In [ ]:
new_chunks = text_splitter.split_documents([new_doc])
new_chunks

In [ ]:
# add new documents to vector store
vectorstore.add_documents(new_chunks)

In [ ]:
vectorstore._collection.count()

In [ ]:
new_question = "what are key concepts in reinforcement learning"
query_rag_lcel(new_question)

# Advanced RAG Techniques: Conversational Memory

In [ ]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# create a prompt that includes the chat history
contextualize_q_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. DO not answer the question,
just reformulate it if needed and otherwise return as it is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ('system', contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ('human', "{input}")
])

In [ ]:
history_aware_retiever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

history_aware_retiever

In [ ]:
qa_system_prompt = """You are an assistant for question-answering tasks
Use the following pieces of retrieved context to answer the question.
If you dont know the answer, just say that you dont know.
Use three sentences maximum and keep the answer concise.

Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [ ]:
conversational_rag_chain = create_retrieval_chain(
    history_aware_retiever,
    question_answer_chain
)

In [ ]:
chat_history = []
result1 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "What is machine learning?"}
)

print(f"Q: What is machine learning?")
print(f"A: {result1['answer']}")

In [ ]:
chat_history.extend([HumanMessage(content="What is machine learning?"),
                     AIMessage(content=result1['answer'])])
chat_history

In [ ]:
result2 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "what is unlabelled data?"}
)

print(f"Q: what is unlabelled data?")
print(f"A: {result2['answer']}")

# Using GROQ

In [ ]:
load_dotenv()

In [ ]:
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

In [ ]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## Load llm : method 1

In [ ]:
llm = ChatGroq(
    model="gemma2-9b-it",
    api_key=os.environ["GROQ_API_KEY"]
)

In [ ]:
llm

## Load llm : method 2

In [ ]:
llm = init_chat_model(
    model="groq:gemma2-9b-it"
)
llm